# Model Training Instructions

1. Upload the locally generated challenge training data archive (`./data/export/dataset.zip`) to Google Drive at the following path:
  ```text
  /mc28/dataset.zip
  ```

2. Enable Notebook GPU acceleration.
  ```text
  Edit > Notebook settings > Hardware accelerator > GPU
  ```

3. Paste the following into Devtools (`Ctrl + Shift + J`) to begin user interaction spoofing. This prevents our Colab session from idling out during training:

  ```javascript
  (() =>
  {
    const click = () =>
    {
      document.querySelector('#top-toolbar > colab-connect-button')
              .shadowRoot
              .querySelector('#connect')
              .click();
    };
  
    const heartbeat = () =>
    {
      click();
      setTimeout(click, 2000);
    };
  
    window.sessionKeepAliveId = setInterval(heartbeat, 3 * 60 * 1000);
  })();
  ```

4. Run all notebook cells to install dependencies and train/export the model.

5. After training/export is complete, cease user interaction spoofing by running the following in Devtools (`Ctrl + Shift + J`):

  ```javascript
  window.sessionKeepAliveId = clearInterval(window.sessionKeepAliveId);
  ```

6. Fetch the generated bundle from Google Drive at: 
  ```text
  /mc28/export.zip
  ```

7. Destroy the runtime and disconnect.

# Mount Google Drive

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

# Configuration

In [ ]:
%env CLASS_COUNT 3
%env MODEL yolov5s

%env EPOCHS 100
%env BATCH_SIZE 64
%env IMAGE_SIZE 416

%env LOG_DIR /mc28/runs
%env DATA_DIR /content/dataset
%env SCRATCH_DIR /content/scratch

# Bootstrap

In [ ]:
%cd /content

!rm -f /mc28
!ln -s /content/gdrive/My\ Drive/mc28/ /mc28

!rm -rf sample_data
!rm -rf $DATA_DIR
!rm -rf $SCRATCH_DIR

!mkdir $DATA_DIR
!mkdir $SCRATCH_DIR
!mkdir -p $LOG_DIR

# Install YOLOv5 and Dependencies



In [ ]:
!git clone --depth 1 --branch v3.0 https://github.com/ultralytics/yolov5
!pip install -qr yolov5/requirements.txt

# Define Notebook Helpers

In [ ]:
%cd yolov5

import json
import torch
import yaml
from os import environ as env

from IPython.display import Image
from utils.general import strip_optimizer

def show(path, width=800, log_dir=env['LOG_DIR']):
  return Image(f'{log_dir}/{path}', width=width)

def optimize(experiment, log_dir=env['LOG_DIR']):
  model_path = f'{log_dir}/{experiment}/weights'
  strip_optimizer(f'{model_path}/best.pt', f'{model_path}/optimized.pt')

# Ensure GPU Acceleration Is Enabled

In [ ]:
assert torch.cuda.is_available()

print(torch.cuda.get_device_properties(0), '\n')

!nvidia-smi

# Prepare Training and Validation Datasets

In [ ]:
%cd /content

!sed "s/nc:\s*[0-9]\+/nc: $CLASS_COUNT/" yolov5/models/$MODEL.yaml > $DATA_DIR/model.yaml

!cp /mc28/dataset.zip .
!unzip -q ./dataset.zip -d $DATA_DIR
!rm ./dataset.zip

%cd yolov5

# Load Tensorboard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir {env['LOG_DIR']}

# Train Model

**NOTE**: The `--cache-images` flag is heavy on RAM but speeds up training considerably. Training with larger datasets (and/or for many epochs) may cause us to exhaust the instance's memory limit. Omit this flag if problems are encountered.

In [ ]:
!python ./train.py           \
  --img-size $IMAGE_SIZE     \
  --batch-size $BATCH_SIZE   \
  --epochs $EPOCHS           \
  --data $DATA_DIR/meta.yaml \
  --cfg $DATA_DIR/model.yaml \
  --weights $MODEL.pt        \
  --logdir $LOG_DIR          \
  --cache-images

In [ ]:
experiment = 'exp0'
env['EXPERIMENT'] = experiment

# Label Metrics

In [ ]:
show(f'{experiment}/labels.png')

# Model Performance

In [ ]:
show(f'{experiment}/results.png', width=1200)

# Debloat Model

In [ ]:
optimize(experiment)

# Export Data

In [ ]:
%cd {env['SCRATCH_DIR']}

!cp $DATA_DIR/model.yaml model.yaml
!cp $LOG_DIR/$EXPERIMENT/weights/optimized.pt model.pt
!cp $LOG_DIR/$EXPERIMENT/labels.png .
!cp $LOG_DIR/$EXPERIMENT/results.png performance.png

with open(f'{env["DATA_DIR"]}/meta.yaml') as input:
  with open('classes.json', 'w') as output:
    json.dump(yaml.safe_load(input)['names'], output)
    output.write('\n')

!zip -rq /mc28/export.zip .

# Disable User Interaction Spoofing

Paste the following into Devtools (`Ctrl + Shift + J`) to cease user interaction spoofing.

```javascript
window.sessionKeepAliveId = clearInterval(window.sessionKeepAliveId);
```